# 迁移学习

在这个 notebook 中，你将学习如何使用预先训练的神经网络解决计算机视觉领域的挑战。具体来说，你使用的网络根据 [ImageNet](http://www.image-net.org/) 中的图像进行训练，你可以在 [torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html) 中找到这些模型。

ImageNet 是一个庞大的数据集，其中有超过一百万张带有标签的图像，来自一千个不同类别。通常，我们使用一种名为卷积层的结构训练深度神经网络。在这里，我并不会深入介绍卷积网络，但如果你感兴趣，可以查看[这个视频](https://www.youtube.com/watch?v=2-Ol7ZB0MmU)。

一旦经过训练，这些模型便能以绝佳表现检测未见过的图像的特征。这种使用预先训练的网络来分析训练集之外的图像的方法被称为迁移学习。在这里，我们将使用迁移学习来训练一个能够以近乎完美的准确性分类猫狗图像的网络。

使用 `torchvision.models`，你可以下载这些预先训练的网络，并用于你的应用中。我们现在将导入 `models`。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms, models

import helper

大多数预先训练的模型要求输入为 224x224 像素的图像。同样地，我们需要匹配训练模型时进行的标准化。每个颜色通道都分别进行了标准化，均值为 `[0.485, 0.456, 0.406]`，标准差为 `[0.229, 0.224, 0.225]`。

In [ ]:
data_dir = '../Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

我们可以载入一个模型，比如 [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5)。现在让我们打印出这个模型的结构，以便了解细节。

In [ ]:
model = models.densenet121(pretrained=True)
model

这个模型主要有两个部分，即特征和分类器。特征部分是一堆卷积层，能作为特征检测器输入分类器中。分类器部分是一个单独的全连接层 `(classifier): Linear(in_features=1024, out_features=1000)`。这个层根据 ImageNet 数据集训练，因此无法解决我们指定的问题。这意味着我们需要替换这个分类器，不过这些特征本身能起到很大的作用。一般来说，我认为预先训练的网络是绝佳的特征检测器，可以作为简单的前馈分类器的输入。

In [ ]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

在构建好模型之后，我们需要训练分类器。然而，现在我们使用的是一个**非常深**的神经网络。如果你还像之前一样试图在 CPU 上训练它，这会耗费相当长的时间。因此，我们将使用 GPU 来进行运算。在 GPU 上，线性代数运算同步进行，这使得运算速度提升了 100x。我们还可以在多个 GPU 上进行训练，这能进一步缩短训练时间。

PyTorch 和其他深度学习框架一样，使用 [CUDA](https://developer.nvidia.com/cuda-zone) 来高效地在 GPU 上计算前向和后向传播。在 PyTorch 中，你可以使用 `model.cuda()` 将模型参数和其他张量转移到 GPU 内存中。当你需要在 PyTorch 之外处理网络的输出时，你也可以使用 `model.cpu()` 再将它们从 GPU 上转移回去。我将分别使用 GPU 和不使用 GPU 进行前向传播和后向传播，好为你展示着两者之间计算速度的差异。

In [ ]:
import time

In [ ]:
for cuda in [False, True]:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    if cuda:
        # Move model parameters to the GPU
        model.cuda()
    else:
        model.cpu()

    for ii, (inputs, labels) in enumerate(trainloader):
        inputs, labels = Variable(inputs), Variable(labels)

        if cuda:
            # Move input and label tensors to the GPU
            inputs, labels = inputs.cuda(), labels.cuda()

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"CUDA = {cuda}; Time per batch: {(time.time() - start)/3:.3f} seconds")

In [ ]:
CUDA = False; Time per batch: 5.443 seconds
CUDA = True; Time per batch: 0.009 seconds


除了手动设置 `cuda` 外，你还可以通过 `cuda = torch.cuda.is_available()` 查看 CUDA 是否可用。

从这里开始，你将自己完成这个模型的训练。这个过程和之前一样，不过现在你的模型更加强大。你的准确率可以轻易达到 95% 以上。

>**练习：** 训练一个预先训练的模型来给猫狗图像分类。你可以在 DenseNet 模型的基础上训练，也可以尝试 ResNet，这也是一个非常优秀的模型。请确保你只训练了分类器，而特征参数部分保留不动。

In [ ]:
# TODO: Train a model with a pre-trained network